In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Land Cover Classification for Urban Expansion Monitoring in Abu Dhabi\n\nThis notebook demonstrates the workflow for land cover classification using Sentinel-2 imagery and machine learning."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Dependencies\n\nImport libraries and set up the environment."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "import ee\n",
    "import rasterio\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import folium\n",
    "from src.data_collection import initialize_gee, get_sentinel_image, export_to_geotiff\n",
    "from src.preprocessing import preprocess_sentinel_image\n",
    "from src.feature_extraction import extract_features\n",
    "from src.classification import LandCoverClassifier\n",
    "from src.visualization import (\n",
    "    plot_classification_map,\n",
    "    create_interactive_map,\n",
    "    plot_feature_importance,\n",
    "    plot_change_detection\n",
    ")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Collection\n\nDownload Sentinel-2 imagery for Abu Dhabi using GEE."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Initialize GEE\n",
    "initialize_gee()\n",
    "\n",
    "# Define Abu Dhabi region of interest\n",
    "abu_dhabi = ee.Geometry.Point([54.37, 24.45]).buffer(10000)  # 10km buffer\n",
    "\n",
    "# Get Sentinel-2 image for 2023\n",
    "image_2023 = get_sentinel_image(abu_dhabi, '2023-01-01', '2023-03-31')\n",
    "\n",
    "# Export to GeoTIFF (run once, then download from Google Drive)\n",
    "task_2023 = export_to_geotiff(image_2023, abu_dhabi, 'sentinel2_abudhabi_2023')\n",
    "print(\"Export task started for 2023 image\")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Preprocessing\n\nPreprocess the downloaded Sentinel-2 imagery."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Preprocess 2023 image\n",
    "preprocess_sentinel_image(\n",
    "    '../data/sentinel2_abudhabi_2023.tif',\n",
    "    '../data/sentinel2_abudhabi_2023_preprocessed.tif'\n",
    ")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Feature Extraction\n\nCalculate NDVI, NDWI, and NDBI from the preprocessed imagery."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Extract features\n",
    "features = extract_features(\n",
    "    '../data/sentinel2_abudhabi_2023_preprocessed.tif',\n",
    "    '../data/sentinel2_abudhabi_2023_features.tif'\n",
    ")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Classification\n\nTrain a Random Forest classifier and perform land cover classification."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Initialize classifier\n",
    "classifier = LandCoverClassifier()\n",
    "\n",
    "# Prepare training data (requires labeled raster)\n",
    "X_train, X_test, y_train, y_test = classifier.prepare_training_data(\n",
    "    '../data/sentinel2_abudhabi_2023_features.tif',\n",
    "    '../data/sentinel2_abudhabi_2023_labels.tif'\n",
    ")\n",
    "\n",
    "# Train model\n",
    "classifier.train(X_train, y_train)\n",
    "\n",
    "# Get feature importance\n",
    "importance = classifier.get_feature_importance()\n",
    "print(\"\\nFeature Importance:\")\n",
    "for feature, score in importance.items():\n",
    "    print(f\"{feature}: {score:.4f}\")\n",
    "\n",
    "# Make predictions\n",
    "predictions = classifier.predict(\n",
    "    '../data/sentinel2_abudhabi_2023_features.tif',\n",
    "    '../data/sentinel2_abudhabi_2023_classified.tif'\n",
    ")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Visualization\n\nGenerate visualizations of the classification results."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import os\n",
    "vis_dir = '../data/visualizations'\n",
    "os.makedirs(vis_dir, exist_ok=True)\n",
    "\n",
    "# Static classification map\n",
    "plot_classification_map(\n",
    "    '../data/sentinel2_abudhabi_2023_classified.tif',\n",
    "    os.path.join(vis_dir, 'classification_map.png')\n",
    ")\n",
    "\n",
    "# Interactive map\n",
    "create_interactive_map(\n",
    "    '../data/sentinel2_abudhabi_2023_classified.tif',\n",
    "    os.path.join(vis_dir, 'interactive_map.html')\n",
    ")\n",
    "\n",
    "# Feature importance plot\n",
    "plot_feature_importance(\n",
    "    importance,\n",
    "    os.path.join(vis_dir, 'feature_importance.png')\n",
    ")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Change Detection\n\nCompare classifications from different years to detect urban expansion."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Change detection map\n",
    "plot_change_detection(\n",
    "    '../data/sentinel2_abudhabi_2017_classified.tif',\n",
    "    '../data/sentinel2_abudhabi_2023_classified.tif',\n",
    "    os.path.join(vis_dir, 'change_detection.png')\n",
    ")"
   ],
   "execution_count": null,
   "outputs": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


NameError: name 'null' is not defined

In [ ]:
import ee
ee.Initialize()

# Use ESA WorldCover 2021 as an example
landcover = ee.ImageCollection("ESA/WorldCover/v100").first().select('Map')

# Define your region of interest (Abu Dhabi)
roi = ee.Geometry.Point([54.37, 24.45]).buffer(10000)  # 10km buffer

# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=landcover,
    description='abudhabi_landcover',
    scale=10,
    region=roi,
    fileFormat='GeoTIFF',
    maxPixels=1e13
)
task.start()
print("Export started. Check your Google Drive after a few minutes.")
